<a href="https://colab.research.google.com/github/skywalker0803r/Wastewater-Biological-Treatment/blob/main/%E6%A0%B9%E6%93%9AcName%E5%92%8CD%E5%80%BC%E5%8D%81%E5%88%86%E4%BD%8D%E6%95%B8%E8%A7%80%E5%AF%9FCOD%E5%B7%AE%E7%95%B0%E9%87%8F%E4%B9%8B%E9%95%B7%E6%A2%9D%E5%9C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [42]:
df1 = pd.read_csv('/content/drive/MyDrive/廢水案子/ALLdataset.csv',index_col = 0)
df1

,site,cindex,cName,classification,confidence,D
date,,,,,,
20210927,T753,101,輪蟲幼蟲,101_輪蟲幼蟲,0.371431,39.638027
20210927,T753,116,絲狀菌,116_絲狀菌,0.996904,18.817467
20210927,T753,100,NaN,100,0.680799,96.061060
20210927,T753,125,膜袋蟲,125_膜袋蟲,0.828820,35.845880
20210927,T753,100,NaN,100,0.696207,77.641043
...,...,...,...,...,...,...
20230216,T753,100,NaN,100,1.000000,175.106430
20230216,T753,100,NaN,100,1.000000,101.303771
20230216,T753,100,NaN,100,1.000000,374.445880


In [43]:
def load_df2():
  path = '/content/drive/MyDrive/專案工作承攬契約書_廢水生物處理/2.20210101-20220430資料(整理).xlsx'
  df = pd.read_excel(path,index_col=0)
  var1 = df.iloc[0,:].values
  var2 = df.iloc[1,:].values
  專有名詞對照表 = pd.concat([pd.Series(var1), pd.Series(var2)], axis=1).T
  專有名詞對照表.index = ['詳細名稱', '英文名稱']
  # 指定第一行為欄位名稱
  df.columns = df.iloc[1]  
  # 去掉前兩行因為是欄位名稱
  df = df.iloc[2:]  
  # cell都轉換成numeric
  for i in df.columns:
    df[i] = pd.to_numeric(df[i],errors='coerce')
  return df,專有名詞對照表
df2,專有名詞對照表 = load_df2()
df2.index = pd.to_datetime(df2.index).strftime('%Y%m%d').astype(df1.index)
df2

日期,PHL-WS-FI7C01_PV,PHL-WS-AT7C03_PV,PHL-WS-AT7C02_PV,PHL-WS-TT7C09_PV,PHL-WS-T752-LEVEL_PV,PHL-WS-FIT7C04_PV,PHL-WS-FI7E01_PV,T-752食微比,PHL-WS-FI7C05_PV,PHL_T751-COD_ppm,...,PHL_A771-COD_ppm,PHL_A771-SS,PHL_A771-PHL,PHL_S7511-PO4,PHL_S7512-PO4,PHL_S7511-NH3,PHL_S7512-NH3,PHL_T751-pH,PHL_S7512-pH,PHL_A771-pH
20210101,30.954139,2.748208,13454.707783,30.738966,82.837878,1322.159831,171.755781,0.074793,7.920856,2366.666667,...,41.000000,12.833333,0.0,21.633334,2.386667,20.550000,0.891667,7.700000,8.083334,8.383333
20210102,31.155222,2.503246,13309.705981,30.492182,82.828679,1299.586141,175.419489,0.085769,7.951025,2696.666667,...,38.333333,12.333333,0.0,22.270000,2.010000,20.601111,0.446667,7.716667,8.000000,8.300000
20210103,30.948144,2.119857,13312.370351,30.441738,82.883022,1275.992065,169.424186,0.085614,7.895686,2710.000000,...,42.333333,14.500000,0.0,24.310001,1.871667,19.680000,0.171667,7.850000,8.016667,8.316667
20210104,33.247575,1.300810,13396.259431,30.483045,82.905756,1298.938517,208.076129,0.097059,7.904003,2860.000000,...,49.666667,16.500000,0.0,22.606667,1.431667,19.422222,0.144444,8.116667,8.083334,8.400000
20210105,32.230836,1.900386,13604.773082,30.641047,82.901661,1334.388753,182.204378,0.095510,7.937889,2903.333333,...,37.666667,14.000000,0.0,21.743333,1.405000,19.848888,0.172222,8.183333,8.016667,8.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220426,39.947576,1.632956,13778.719592,40.284236,82.138722,1534.251369,200.641541,0.116615,-0.033719,2213.560000,...,49.140000,6.461538,0.0,16.188000,2.337000,34.438888,0.237222,7.633846,7.810000,8.310000
20220427,38.524569,1.619828,13838.553847,40.583724,82.098759,1532.456603,190.895455,0.115005,-0.033638,2271.880000,...,50.220000,6.500000,0.0,15.324000,2.151000,34.005555,0.233889,7.765000,7.760000,8.285000
20220428,36.809783,1.645253,13813.169294,40.412546,82.117266,1526.493705,196.933991,0.112299,-0.033809,2330.200000,...,51.300000,7.500000,0.0,14.460000,1.965000,33.572222,0.230556,7.815000,7.733333,8.263333
20220429,35.945983,1.758189,13622.316753,39.928740,82.106717,1526.142494,188.211287,0.112004,-0.032991,2388.520000,...,52.380000,8.500000,0.0,13.596000,1.779000,33.138889,0.227222,7.685000,7.780000,8.270000


In [44]:
專有名詞對照表.T

,詳細名稱,英文名稱
0,T752入料量,PHL-WS-FI7C01_PV
1,T752溶氧,PHL-WS-AT7C03_PV
2,T752導電度計,PHL-WS-AT7C02_PV
3,T752溫度,PHL-WS-TT7C09_PV
4,T752 液位,PHL-WS-T752-LEVEL_PV
5,T752曝氣量,PHL-WS-FIT7C04_PV
6,放流水量,PHL-WS-FI7E01_PV
7,T-752食微比,T-752食微比
8,T752排泥量,PHL-WS-FI7C05_PV
9,T751COD濃度,PHL_T751-COD_ppm


In [45]:
df = df1.join(df2)
df

,site,cindex,cName,classification,confidence,D,PHL-WS-FI7C01_PV,PHL-WS-AT7C03_PV,PHL-WS-AT7C02_PV,PHL-WS-TT7C09_PV,...,PHL_A771-COD_ppm,PHL_A771-SS,PHL_A771-PHL,PHL_S7511-PO4,PHL_S7512-PO4,PHL_S7511-NH3,PHL_S7512-NH3,PHL_T751-pH,PHL_S7512-pH,PHL_A771-pH
20210927,T753,101,輪蟲幼蟲,101_輪蟲幼蟲,0.371431,39.638027,32.166868,0.698399,14051.069547,38.956273,...,43.166667,10.0,0.0,20.266667,3.365,20.572223,0.303333,7.883333,8.233333,8.433333
20210927,T753,116,絲狀菌,116_絲狀菌,0.996904,18.817467,32.166868,0.698399,14051.069547,38.956273,...,43.166667,10.0,0.0,20.266667,3.365,20.572223,0.303333,7.883333,8.233333,8.433333
20210927,T753,100,NaN,100,0.680799,96.061060,32.166868,0.698399,14051.069547,38.956273,...,43.166667,10.0,0.0,20.266667,3.365,20.572223,0.303333,7.883333,8.233333,8.433333
20210927,T753,125,膜袋蟲,125_膜袋蟲,0.828820,35.845880,32.166868,0.698399,14051.069547,38.956273,...,43.166667,10.0,0.0,20.266667,3.365,20.572223,0.303333,7.883333,8.233333,8.433333
20210927,T753,100,NaN,100,0.696207,77.641043,32.166868,0.698399,14051.069547,38.956273,...,43.166667,10.0,0.0,20.266667,3.365,20.572223,0.303333,7.883333,8.233333,8.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230216,T753,100,NaN,100,1.000000,175.106430,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20230216,T753,100,NaN,100,1.000000,101.303771,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20230216,T753,100,NaN,100,1.000000,374.445880,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20230216,T753,100,NaN,100,1.000000,63.488342,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
need_cols = ['cName','D','PHL_T751-COD_ppm','PHL_A771-COD_ppm']
df = df.loc[:,need_cols]
df = df.dropna(axis=0)
df

,cName,D,PHL_T751-COD_ppm,PHL_A771-COD_ppm
20210927,輪蟲幼蟲,39.638027,2776.666667,43.166667
20210927,絲狀菌,18.817467,2776.666667,43.166667
20210927,膜袋蟲,35.845880,2776.666667,43.166667
20210927,絲狀菌,32.625321,2776.666667,43.166667
20210927,絲狀菌,22.131402,2776.666667,43.166667
...,...,...,...,...
20220429,太陽蟲,30.405082,2388.520000,52.380000
20220429,絲狀菌,31.968808,2388.520000,52.380000
20220429,太陽蟲,90.186645,2388.520000,52.380000
20220429,絲狀菌,25.094651,2388.520000,52.380000


In [47]:
cName_list = list(set(df['cName'].unique())-set(['絲狀菌']))
cName_list

['變形蟲',
 '螺旋菌',
 '太陽蟲',
 '輪蟲幼蟲',
 '表殼蟲',
 '膜袋蟲',
 '腔輪蟲',
 '輪蟲類',
 '變形蟲2',
 '單領鞭毛蟲',
 '單鐮蟲',
 '尾絲蟲',
 '漫遊蟲',
 '滴蟲',
 '斜管蟲']

In [48]:
def transform_data(df,cName):
  # select_data
  d = df.loc[df['cName']==cName]
  # 10day resample
  d.index = pd.to_datetime(d.index, format='%Y%m%d')
  d = d.resample('10D').mean()
  # add cName
  d['cName'] = cName
  # cacluate COD diffrence
  d['COD差異量'] = d['PHL_T751-COD_ppm']-d['PHL_A771-COD_ppm']
  # cluster by D
  quantiles = [i / 10.0 for i in range(9)]
  d_quantiles = d['D'].quantile(quantiles)
  def label_function(value):
    for i, quantile in enumerate(d_quantiles):
      if value <= quantile:
        return i
    return len(d_quantiles)  
  d['label'] = d['D'].apply(label_function)
  return d

In [49]:
temp = pd.DataFrame()
for i in cName_list:
  temp = temp.append(transform_data(df,i))
df = temp
df

,D,PHL_T751-COD_ppm,PHL_A771-COD_ppm,cName,COD差異量,label
2021-09-27,137.056732,2659.685714,41.726667,變形蟲,2617.959048,1
2021-10-07,174.507197,2519.392523,39.989097,變形蟲,2479.403427,8
2021-10-17,182.241932,2836.933333,40.879259,變形蟲,2796.054074,9
2021-10-27,161.221834,2469.634081,43.849359,變形蟲,2425.784722,7
2021-11-06,161.820670,2204.970760,43.391082,變形蟲,2161.579678,7
...,...,...,...,...,...,...
2022-03-16,68.478895,1601.609023,41.850877,斜管蟲,1559.758145,8
2022-03-26,82.027830,2099.263374,36.876543,斜管蟲,2062.386831,9
2022-04-05,53.489667,2444.708333,36.574074,斜管蟲,2408.134259,2
2022-04-15,50.205675,2462.857143,34.857143,斜管蟲,2428.000000,1


In [50]:
df = df.sort_values(by='label')
df

,D,PHL_T751-COD_ppm,PHL_A771-COD_ppm,cName,COD差異量,label
2022-04-25,33.852765,2297.800000,50.700000,螺旋菌,2247.100000,0
2022-04-15,27.319595,2462.857143,34.857143,變形蟲2,2428.000000,0
2022-04-05,31.089110,2104.611111,34.444444,單領鞭毛蟲,2070.166667,0
2021-09-27,34.833270,2642.007042,42.123631,膜袋蟲,2599.883412,0
2021-09-27,54.124991,2562.592593,41.981481,單鐮蟲,2520.611111,0
...,...,...,...,...,...,...
2022-04-05,72.616772,2240.443452,35.476190,單鐮蟲,2204.967262,9
2021-12-06,43.062938,2179.073782,40.951687,膜袋蟲,2138.122095,9
2022-03-16,75.915066,1621.518519,42.000000,單鐮蟲,1579.518519,9
2022-03-16,95.302555,1553.658319,41.850604,漫遊蟲,1511.807714,9


In [51]:
import plotly.express as px
fig = px.bar(df,x="cName", y="COD差異量", color="label", title="根據'cName'和'D'的十分位數觀察'COD差異量'之長條圖")
fig.show()

In [52]:
import plotly.graph_objects as go
fig1 = go.Figure()  # 生成一个Figure对象

# 通过循环在对象上添加4个轨迹trace
for cName in df.cName.unique():
    fig1.add_trace(go.Violin(
        x=df["cName"][df["cName"] == cName],
        y=df["COD差異量"][df["cName"] == cName],
        name=cName,
        box_visible=True,
        meanline_visible=True,
        
       ))
fig1.update_layout(title="cName對COD差異量")  # 添加标题
fig1.show()
#我怎麼在fig1加上title

In [53]:
import plotly.graph_objects as go
fig1 = go.Figure()  # 生成一个Figure对象

# 通过循环在对象上添加4个轨迹trace
for cName in df.cName.unique():
    fig1.add_trace(go.Violin(
        x=df["cName"][df["cName"] == cName],
        y=df["label"][df["cName"] == cName],
        name=cName,
        box_visible=True,
        meanline_visible=True
       ))
fig1.update_layout(title="cName對label(根據D值分十分位數)")  # 添加标题
fig1.show()